# Define S3 client

In [ ]:
import os

s3endpoint = os.environ['s3endpoint']
# s3bucket = os.environ['s3bucket']
mount_path = os.environ['mount_path']

In [7]:
import os
import boto3
from botocore.client import Config

# s3_client = boto3.client('s3',
#         # endpoint_url='https://minio-api-test02.apps.rhods-internal.61tk.p1.openshiftapps.com',
#         endpoint_url='https://minio-api-test03.apps.rhods-internal.61tk.p1.openshiftapps.com',
#         aws_access_key_id='minio',
#         aws_secret_access_key='minio123',
#         config=Config(signature_version='s3v4'),
#         region_name='us-east-1')

# s3endpoint='https://minio-api-test02.apps.rhods-internal.61tk.p1.openshiftapps.com',
# s3endpoint = "https://minio-api-demo-ai.apps.cluster-ssfg4.dynamic.opentlc.com"
# s3bucket = "production"
s3bucket = "pre-production"


s3_client = boto3.client('s3',
        endpoint_url=s3endpoint,
        aws_access_key_id='minio',
        aws_secret_access_key='minio123',
        config=Config(signature_version='s3v4'),
        region_name='us-east-1')

      

# Clear bucket

In [8]:
  
# objects = s3_client.list_objects_v2(Bucket='production')
objects = s3_client.list_objects_v2(Bucket=s3bucket)



# print(objects)

if "Contents" in objects:
    print("removing contents")
    remove = {"Objects": []}
    for obj in objects['Contents']:
        print(obj)
        remove['Objects'].append({'Key':obj['Key']})
    s3_client.delete_objects(Bucket=s3bucket, Delete=remove)

else:
    print("there are NO contents")

removing contents
{'Key': 'models/tea_model_b64/1/fingerprint.pb', 'LastModified': datetime.datetime(2023, 12, 14, 14, 8, 7, 725000, tzinfo=tzlocal()), 'ETag': '"88af44942145a6db3c820ee012697b6e"', 'Size': 55, 'StorageClass': 'STANDARD'}
{'Key': 'models/tea_model_b64/1/saved_model.pb', 'LastModified': datetime.datetime(2023, 12, 14, 14, 8, 5, 679000, tzinfo=tzlocal()), 'ETag': '"aeb7580fe3ac50ce925ab45366a49b2e"', 'Size': 4241279, 'StorageClass': 'STANDARD'}
{'Key': 'models/tea_model_b64/1/variables/variables.data-00000-of-00001', 'LastModified': datetime.datetime(2023, 12, 14, 14, 8, 8, 559000, tzinfo=tzlocal()), 'ETag': '"c7a55f4037d1fca4780f337ae9d84947-2"', 'Size': 9217730, 'StorageClass': 'STANDARD'}
{'Key': 'models/tea_model_b64/1/variables/variables.index', 'LastModified': datetime.datetime(2023, 12, 14, 14, 8, 7, 801000, tzinfo=tzlocal()), 'ETag': '"01a9b50e250d25d47f43cf3211ee7dfe"', 'Size': 15472, 'StorageClass': 'STANDARD'}


# Upload model

In [9]:
# path = "models/tea_model_b64"
path = mount_path+"models/tea_model_b64"

for subdir, dirs, files in os.walk(path):
    for file in files:
        full_path = os.path.join(subdir, file)
        s3_path = os.path.join(subdir.removeprefix(mount_path), file)
        print(s3_path)
        s3_client.upload_file(full_path, s3bucket, s3_path)
        

models/tea_model_b64/1/saved_model.pb
models/tea_model_b64/1/fingerprint.pb
models/tea_model_b64/1/variables/variables.index
models/tea_model_b64/1/variables/variables.data-00000-of-00001
